# Jerk Limited Motion Control
Adham Elarabawy  
June 2021

## Source
All referenced implementations can be found in [my motion repository](https://github.com/adham-elarabawy/motion). 

All jerk-limited implementations are based on [this textbook](http://home.elka.pw.edu.pl/~ptrojane/books/Trajectory_Planning_for_Automatic_Machines_and_Robots.pdf).

## Introduction

Currently, all simple motion planning that occurs on the Fuse machines is orchestrated by a trapezoidal/triangular motion planner, which effectively describes the resulting velocity profile. 

**Note:** *From here on out, I will be referring to triangular and trapezoidal velocity profiles as trapezoidal for concision. The main difference is that a triangular profile is used when the maximum velocity is not reached based on the given parameters, so the trajectory must decelerate before hitting a velocity plateau.*

This trapezoidal velocity profile is generated via setting maximum acceleration/deceleration during certain periods, and then integrating the resultant acceleration to generate the corresponding velocity/position profiles. These profiles can then be discretized and sampled to generate low-level stepper motor move commands.

Here is an example of a trapezoidal velocity profile:

![Trapezoidal Velocity Profile](https://github.com/adham-elarabawy/motion/blob/main/media/trapezoidal.png?raw=true)
The above trajectory was generated by [this code](https://github.com/adham-elarabawy/motion/blob/main/examples/run_trapezoidal.py).

### What's the problem?

As you can see in the below diagram, there are points of instantaneous change in acceleration (circled in red), which correlate to extremely large jerk values -- which result in unwanted system properties (vibrations & more time needed to settle).

![Trapezoidal Velocity Profile Problems](https://github.com/adham-elarabawy/motion/blob/main/media/trapezoidal_diagram.png?raw=true)

To solve this problem, we set a jerk limit and effectively perform the same algorithm as before, integrating each previous profile to generate acceleration from jerk, velocity from acceleration, and position from velocity. This explanation is greatly simplified, since it completely glosses over how we actually know how long we need to accelerate, decelerate, etc. This will be explained in-depth later in this note, but it is important to first illustrate the fundamental theory.

Here is an example of a jerk-limited velocity profile:

![Jerk Limited Velocity Profile](https://github.com/adham-elarabawy/motion/blob/main/media/jerklimited.png?raw=true)
The above trajectory was generated by [this code](https://github.com/adham-elarabawy/motion/blob/main/examples/run_jerk_limited.py).

### A quick concession + rebuttal
Before we can fully dive into the math behind the jerk limited trajectory generation, I want to emphasize an important point with regards to time optimality and compromises.

Clearly, by reducing the portions in which we are allowing our system to travel at maximum acceleration, we are generating a trajectory that is comparatively slower than a simple trapezoidal velocity profile. Intuitively, this makes sense -- and it should since it is true.

However, it is important to bring attention to the role of system dynamics within the context of trajectory planning. Sure, generating a jerk-limited trajectory with the equivalent parameters as a trapezoidal trajectory will give you a trajectory that takes longer to execute. The key insight is in noting that a jerk-limited trajectory theoretically allows you to **increase** your maximum acceleration and velocity, since those are usually driven by experiments on system stability anyways. So, a jerk-limited approach may actually result in faster trajectories, since it will allow you to be more aggressive with the values you pick for max acceleration and velocity.  

Additionally, the performance benefits of jerk-limited trajectories are well documented: theoretically less mechanical stress on your system, seemingly softer moves (without any "jerky" moves), etc.

## Technical Implementation

The technical implementation that I am describing here is thoroughly and formally explained in [this textbook](http://home.elka.pw.edu.pl/~ptrojane/books/Trajectory_Planning_for_Automatic_Machines_and_Robots.pdf#page=88) starting on page 88-110. However, I will spare you the verbosity and will instead walk you through my python implementation of the described jerk-limited trajectory generation algorithm. You can find my raw python source code for the following jerk-limited trajectory generation [here](https://github.com/adham-elarabawy/motion/blob/main/engine/jerk_limited.py).

#### Imports
First, we import the required dependencies

In [3]:
# computation dependencies
import numpy as np

# visualization dependencies
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#### Initial parameters
Here, we set the initial conditions. If we were making this a function, these are the parameters we would pass in for each generated trajectory.

In [6]:
_q0 = 0     # initial position (mm)
_q1 = 10    # final position (mm)
_v0 = 1     # initial velocity (mm/s)
_v1 = 0     # final velocity (mm/s)
_v_max = 50 # maximum velocity (mm/s)
_a_max = 10 # maximum acceleration (mm/s^2)
_j_max = 15 # maximum jerk (mm/s^3)
alpha = 0.9 # iterative scaling constant (will be described in detail later.)

#### Parameter cleanup
Here, we perform a cleanup on our input parameters such that we allow reversed inputs (where the initial position is larger than the final position).

This is mitigated by the omega constant which is simply defined as: $$ \omega =  sign({\_q}_1 - {\_q}_0) $$

In [7]:
omega = np.sign(_q1 - _q0)

_v_min = -_v_max
_a_min = -_a_max
_j_min = -_j_max

q0 = omega * _q0
q1 = omega * _q1
v0 = omega * _v0
v1 = omega * _v1
v_max = ((omega + 1) / 2) * _v_max + ((omega - 1) / 2) * _v_min
v_min = ((omega + 1) / 2) * _v_min + ((omega - 1) / 2) * _v_max
a_max = ((omega + 1) / 2) * _a_max + ((omega - 1) / 2) * _a_min
a_min = ((omega + 1) / 2) * _a_min + ((omega - 1) / 2) * _a_max
j_max = ((omega + 1) / 2) * _j_max + ((omega - 1) / 2) * _j_min
j_min = ((omega + 1) / 2) * _j_min + ((omega - 1) / 2) * _j_max

Adding/subtracting 1 to the sign and then dividing by 2 is a nice trick to easily invert between maximum and minimum parameters based on the sign of omega. The reason we do this instead of just multipying our parameters by -1 is because we want to support the ability to have asymmetric jerk/acceleration/velocity. For instance, if we want the magnitude of our minimum and maximum velocities to be different values. More explicitly, we want to be able to support the cases where $$ \_v\_min \neq -\_v\_max $$ (and the same for acceleration/jerk).

#### Checking feasibility
Now that we've properly adjusted the signs of our input parameters for consistency, we now have proper constraints on the maximum values of jerk, acceleration, and velocity. Before we go about computing the durations of each phase of our trajectory to obey those constraints, it is first necessary to verify that the trajectory can be computed.

**When would a trajectory not be able to be computed?**

**If the desired displacement is small with respect to the difference between initial and final velocities v0 & v1, then the constraints on jerk and acceleration determine whether or not the trajectory can be performed.**


We compute an intermediary variable $T_j^*$ as $$T_j^* = min\{\sqrt{\frac{|v_1 - v_0|}{j_{max}}}, \frac{a_{max}}{j_{max}}\}$$
If $ T_j^* = \frac{a_{max}}{j_{max}} $, then we know that $\frac{a_{max}}{j_{max}} \leq \sqrt{\frac{|v_1 - v_0|}{j_{max}}}$, which tells us that our initial and final velocity inputs *do not* constrain our ability to reach maximum acceleration. AKA, our acceleration reaches a maximum value and a zero jerk phase may exist (refer to phase diagram above).

<font color='red'>This does NOT yet mean our trajectory is entirely feasible, it only verifies that the velocity inputs nicely allow us to reach maximum acceleration with NO iteration.</font>

In this case, we have one more check to perform to validate that we can compute the trajectory. Namely:
$$q_1 - q_0 > \frac{1}{2} (v_0 + v_1) (T_j^* + \frac{|v_1 - v_0|}{a_{max}})$$

If $ T_j^* < \frac{a_{max}}{j_{max}} $, then we know that $\frac{a_{max}}{j_{max}} > \sqrt{\frac{|v_1 - v_0|}{j_{max}}}$, which tells us we don't reach our maximum acceleration during the trajectory.

<font color='red'>This does NOT yet mean our trajectory isn't feasible, it just means that we won't be able to reach maximum acceleration values. Rather, we will have to iteratively reduce our acceleration until the trajectory becomes achievable.</font> The intuition here is that we always want to try to go at the maximum acceleration when we can. If that makes the trajectory impossible, then we iteratively lower the max acceleration we try to reach until the numbers all like each other. In a later step, this is where the alpha parameter will become relevant. The alpha parameter is the reduction scaling factor we multiply the max acceleration by in each iteration to hopefully achieve a feasible trajectory. 